In [1]:
import pandas as pd

df = pd.read_csv(
    'dataset/CollegeBasketballPlayers2009-2021.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23904\2874855951.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [91]:
df.shape

(61061, 66)

In [2]:
# renaming attribute
df = df.rename(columns={'Unnamed: 64': 'role_position'})

In [93]:
df.columns

Index(['player_name', 'team', 'conf', 'GP', 'Min_per', 'Ortg', 'usg', 'eFG',
       'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA',
       'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'Rec Rank', 'ast/tov', 'rimmade',
       'rimmade+rimmiss', 'midmade', 'midmade+midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade',
       'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'role_position', 'Unnamed: 65'],
      dtype='object')

In [3]:
df['role_position'].value_counts()

role_position
Wing G        12455
PF/C          10220
Combo G        8966
C              8121
Scoring PG     5837
Wing F         4923
Stretch 4      3191
Pure PG        2664
Name: count, dtype: int64

In [5]:
# count number of picks that are not NaN
df['pick'].count()

np.int64(1435)

In [7]:
df['player_name'].nunique()

25719

In [95]:
keep_cols = [
    # ===== IDENTIFICATION =====
    "player_name",  # For identification only

    # ===== CONTEXT FEATURES =====
    "conf",         # Conference (competition level)
    "yr",           # Year in school (experience)
    "role_position",  # Position played

    # ===== PLAYING TIME & USAGE =====
    "Min_per",      # Minutes per game (opportunity/trust)
    "usg",          # Usage rate (how much offense runs through player)
    "GP",           # Games played (availability)

    # ===== EFFICIENCY METRICS (MOST IMPORTANT) =====
    "eFG",          # Effective field goal % (shooting efficiency)
    "TS_per",       # True shooting % (overall scoring efficiency)
    "FT_per",       # Free throw % (skill indicator)
    "twoP_per",     # Two-point shooting %
    "TP_per",       # Three-point shooting %
    "ast/tov",      # Assist-to-turnover ratio

    # ===== ADVANCED METRICS =====
    "Ortg",         # Offensive rating (points per 100 possessions)
    "bpm",          # Box Plus/Minus (overall impact)
    "obpm",         # Offensive Box Plus/Minus
    "dbpm",         # Defensive Box Plus/Minus
    "adjoe",        # Adjusted offensive efficiency
    "drtg",         # Defensive rating
    "adrtg",        # Adjusted defensive rating

    # ===== PERCENTAGE STATS (SITUATIONAL) =====
    "ORB_per",      # Offensive rebounding %
    "DRB_per",      # Defensive rebounding %
    "AST_per",      # Assist %
    "TO_per",       # Turnover %
    "blk_per",      # Block %
    "stl_per",      # Steal %
    "ftr",          # Free throw rate (ability to get to line)

    # ===== SHOOTING BREAKDOWN =====
    "rimmade/(rimmade+rimmiss)",           # Shooting at rim
    "midmade/(midmade+midmiss)",           # Mid-range shooting
    "dunksmade/(dunksmade+dunksmiss)",     # Dunk rate

    # ===== COUNTING STATS (PRODUCTION) =====
    "pts",          # Points per game
    "treb",         # Total rebounds
    "ast",          # Assists
    "stl",          # Steals
    "blk",          # Blocks

    "pick"
]


df = df[[c for c in keep_cols if c in df.columns]]
df.shape

(61061, 36)

In [96]:
df["pick"] = df["pick"].fillna("Undrafted")


def categorize_pick(p):
    if p == "Undrafted":
        return "Undrafted"
    try:
        p = int(p)
        if p <= 14:
            return "Lottery"
        elif p <= 30:
            return "1st Round"
        elif p <= 60:
            return "2nd Round"
        else:
            return "Undrafted"  # safeguard
    except:
        return "Undrafted"


df["pick_category"] = df["pick"].apply(categorize_pick)

In [97]:
df['pick_category'].value_counts()

pick_category
Undrafted    59626
2nd Round      812
1st Round      361
Lottery        262
Name: count, dtype: int64

In [ ]:
# Add these before your current preprocessing
def add_draft_specific_features(df):
    df_enhanced = df.copy()

    # 1. Elite performance indicators (top 10% thresholds)
    df_enhanced['elite_scorer'] = (
        df['pts'] > df['pts'].quantile(0.9)).astype(int)
    df_enhanced['elite_efficiency'] = (
        df['TS_per'] > df['TS_per'].quantile(0.9)).astype(int)
    df_enhanced['elite_impact'] = (
        df['bpm'] > df['bpm'].quantile(0.9)).astype(int)

    # 2. Size + skill combination (NBA-ready indicators)
    df_enhanced['versatile_big'] = (
        (df['role_position'].isin(['PF', 'C'])) &
        (df['TP_per'] > 0.3)  # Big who can shoot 3s
    ).astype(int)

    # 3. Usage + efficiency (can handle big role efficiently)
    df_enhanced['high_usage_efficient'] = (
        (df['usg'] > df['usg'].quantile(0.8)) &
        (df['TS_per'] > 0.55)
    ).astype(int)

    # 4. Power conference performance
    power_confs = ['ACC', 'Big 12', 'Big Ten', 'SEC', 'Pac-12', 'Big East']
    df_enhanced['power_conf_star'] = (
        (df['conf'].isin(power_confs)) &
        (df['pts'] > 15)  # Star in power conference
    ).astype(int)

    return df_enhanced


# Apply before train-test split
df_enhanced = add_draft_specific_features(df)

In [99]:
df_enhanced.shape

(61061, 43)

In [ ]:
from sklearn.model_selection import train_test_split


# Separate target
X = df_enhanced.drop(columns=["pick", "player_name","pick_category"])  # features
y = df_enhanced["pick_category"]  # target

# Final pipeline (ready for any model)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print("Preprocessing pipeline ready. X_train shape:", X_train.shape)

✅ Preprocessing pipeline ready. X_train shape: (48848, 40)


In [103]:
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


# -----------------------
# 2. Preprocessing
# -----------------------
numeric_features = X.select_dtypes(
    include=["int64", "float64"]).columns.tolist()
categorical_features = [ "conf", "yr", "role_position"]

# Imputers
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

# Fit imputers
X_train_num = num_imputer.fit_transform(X_train[numeric_features])
X_test_num = num_imputer.transform(X_test[numeric_features])

X_train_cat = cat_imputer.fit_transform(X_train[categorical_features])
X_test_cat = cat_imputer.transform(X_test[categorical_features])

# Scale numeric
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

# Encode categorical
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

# Concatenate processed features
X_train_processed = np.hstack([X_train_num, X_train_cat])
X_test_processed = np.hstack([X_test_num, X_test_cat])

In [105]:
import xgboost as xgb
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_sample_weight

# -----------------------
# 3. Stage 1 Labels (Drafted vs Undrafted)
# -----------------------
y_stage1 = (y_train != "Undrafted").astype(int)   # 0 = Undrafted, 1 = Drafted
y_stage1_test = (y_test != "Undrafted").astype(int)

# -----------------------
# 4. Handle Imbalance (SMOTE)
# -----------------------
smote = SMOTE(random_state=42, sampling_strategy=0.8, k_neighbors=3)
X_train_stage1, y_stage1_bal = smote.fit_resample(X_train_processed, y_stage1)

sample_weights = compute_sample_weight('balanced', y_stage1_bal)

# -----------------------
# 5. Train Stage 1 Classifier
# -----------------------
scale_pos_weight = (len(y_stage1) - sum(y_stage1)) / sum(y_stage1)

xgb_stage1 = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    class_weight='balanced',
    # scale_pos_weight=3,
    max_depth=6,
    learning_rate=0.1,
    n_estimators=200,
    # scale_pos_weight=scale_pos_weight,
    random_state=42,
    use_label_encoder=False
)

xgb_stage1.fit(X_train_stage1, y_stage1_bal, sample_weight=sample_weights)
y_stage1_pred = xgb_stage1.predict(X_test_processed)

print("\n=== Stage 1 Report (Drafted vs Undrafted) ===")
print(classification_report(y_stage1_test, y_stage1_pred))



c:\Users\ASUS\Documents\Side Missions\scout-ai\backend\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:25:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "class_weight", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== Stage 1 Report (Drafted vs Undrafted) ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     11905
           1       0.57      0.56      0.56       308

    accuracy                           0.98     12213
   macro avg       0.78      0.77      0.78     12213
weighted avg       0.98      0.98      0.98     12213



In [119]:
# =============================================
# ENSEMBLE OF BEST MODELS
# =============================================

from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# Create ensemble of top performers
ensemble = VotingClassifier(
    estimators=[
        ('catboost', CatBoostClassifier(class_weights=[1, 8], verbose=False)),
        ('lightgbm', LGBMClassifier(class_weight='balanced', verbose=-1)),
        ('rf', RandomForestClassifier(class_weight='balanced', n_jobs=-1))
    ],
    voting='soft'  # Use probability averaging
)

ensemble.fit(X_train_stage1, y_stage1_bal)
y_ensemble_pred = ensemble.predict(X_test_processed)

print("\n🚀 ENSEMBLE Results:")
print(classification_report(y_stage1_test, y_ensemble_pred))


🚀 ENSEMBLE Results:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     11905
           1       0.53      0.63      0.57       308

    accuracy                           0.98     12213
   macro avg       0.76      0.81      0.78     12213
weighted avg       0.98      0.98      0.98     12213



c:\Users\ASUS\Documents\Side Missions\scout-ai\backend\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [121]:
# =============================================
# SAVE MODEL AND PREPROCESSORS FOR DEPLOYMENT
# =============================================
import pickle
import joblib
from datetime import datetime

# Create models directory
import os
os.makedirs('models', exist_ok=True)

# Save the trained ensemble model
model_filename = f'models/nba_draft_ensemble_model_{datetime.now().strftime("%Y%m%d")}.pkl'
joblib.dump(ensemble, model_filename)

# Save all preprocessors (CRITICAL for production)
preprocessors = {
    'num_imputer': num_imputer,
    'cat_imputer': cat_imputer,
    'scaler': scaler,
    'encoder': encoder,
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'feature_columns': list(X.columns),  # Original feature names
    'smote': smote  # In case you need to understand the training process
}

preprocessor_filename = f'models/nba_draft_preprocessors_{datetime.now().strftime("%Y%m%d")}.pkl'
joblib.dump(preprocessors, preprocessor_filename)

print(f"✅ Model saved: {model_filename}")
print(f"✅ Preprocessors saved: {preprocessor_filename}")

✅ Model saved: models/nba_draft_ensemble_model_20250925.pkl
✅ Preprocessors saved: models/nba_draft_preprocessors_20250925.pkl


In [122]:
# =============================================
# PRODUCTION-READY NBA DRAFT PREDICTOR CLASS
# =============================================
import pandas as pd
import numpy as np
import joblib
from typing import Dict, List, Union


class NBADraftPredictor:
    """
    NBA Draft Prospect Predictor
    Predicts if a college basketball player will be drafted to NBA
    """

    def __init__(self, model_path: str, preprocessor_path: str):
        """Load trained model and preprocessors"""
        self.model = joblib.load(model_path)
        self.preprocessors = joblib.load(preprocessor_path)

        # Extract preprocessors
        self.num_imputer = self.preprocessors['num_imputer']
        self.cat_imputer = self.preprocessors['cat_imputer']
        self.scaler = self.preprocessors['scaler']
        self.encoder = self.preprocessors['encoder']
        self.numeric_features = self.preprocessors['numeric_features']
        self.categorical_features = self.preprocessors['categorical_features']
        self.feature_columns = self.preprocessors['feature_columns']

        print("✅ NBA Draft Predictor loaded successfully!")
        print(f"   Model: {type(self.model).__name__}")
        print(f"   Expected features: {len(self.feature_columns)}")

    def add_draft_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add engineered features used during training"""
        df_enhanced = df.copy()

        # Elite performance indicators (top 10% thresholds from training data)
        df_enhanced['elite_scorer'] = (df['pts'] > 20.0).astype(
            int)  # Approximate threshold
        df_enhanced['elite_efficiency'] = (df['TS_per'] > 0.65).astype(int)
        df_enhanced['elite_impact'] = (df['bpm'] > 8.0).astype(int)

        # Size + skill combination
        df_enhanced['versatile_big'] = (
            (df['role_position'].isin(['PF', 'C'])) &
            (df['TP_per'] > 0.3)
        ).astype(int)

        # Usage + efficiency
        df_enhanced['high_usage_efficient'] = (
            (df['usg'] > 25.0) &  # Approximate 80th percentile
            (df['TS_per'] > 0.55)
        ).astype(int)

        # Power conference performance
        power_confs = ['ACC', 'Big 12', 'Big Ten', 'SEC', 'Pac-12', 'Big East']
        df_enhanced['power_conf_star'] = (
            (df['conf'].isin(power_confs)) &
            (df['pts'] > 15)
        ).astype(int)

        return df_enhanced

    def preprocess_data(self, df: pd.DataFrame) -> np.ndarray:
        """Preprocess input data exactly like training data"""

        # Add engineered features
        df_processed = self.add_draft_features(df)

        # Ensure all required columns are present
        missing_cols = set(self.feature_columns) - set(df_processed.columns)
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Select only the features used in training
        df_processed = df_processed[self.feature_columns]

        # Separate numeric and categorical
        df_num = df_processed[self.numeric_features]
        df_cat = df_processed[self.categorical_features]

        # Apply same preprocessing as training
        X_num = self.num_imputer.transform(df_num)
        X_cat = self.cat_imputer.transform(df_cat)

        # Scale and encode
        X_num = self.scaler.transform(X_num)
        X_cat = self.encoder.transform(X_cat)

        # Combine features
        X_processed = np.hstack([X_num, X_cat])

        return X_processed

    def predict_draft_probability(self, player_data: Union[Dict, pd.DataFrame]) -> Dict:
        """
        Predict draft probability for a player or players

        Args:
            player_data: Dict with player stats or DataFrame with multiple players

        Returns:
            Dict with prediction results
        """
        # Convert dict to DataFrame if needed
        if isinstance(player_data, dict):
            df = pd.DataFrame([player_data])
            single_player = True
        else:
            df = player_data.copy()
            single_player = False

        try:
            # Preprocess data
            X_processed = self.preprocess_data(df)

            # Get predictions and probabilities
            predictions = self.model.predict(X_processed)
            probabilities = self.model.predict_proba(
                X_processed)[:, 1]  # Probability of being drafted

            # Prepare results
            results = {
                'predictions': predictions.tolist(),
                'draft_probabilities': probabilities.tolist(),
                'draft_decisions': ['Drafted' if pred == 1 else 'Undrafted' for pred in predictions]
            }

            # Add player names if available
            if 'player_name' in df.columns:
                results['player_names'] = df['player_name'].tolist()

            # If single player, return simplified format
            if single_player:
                return {
                    'player_name': df['player_name'].iloc[0] if 'player_name' in df.columns else 'Unknown',
                    'draft_probability': float(probabilities[0]),
                    'prediction': 'Drafted' if predictions[0] == 1 else 'Undrafted',
                    'confidence': 'High' if probabilities[0] > 0.7 or probabilities[0] < 0.3 else 'Medium'
                }

            return results

        except Exception as e:
            return {'error': f"Prediction failed: {str(e)}"}


# Save the predictor class
predictor_code = '''
# Copy the NBADraftPredictor class code here for external use
'''

with open('models/nba_draft_predictor.py', 'w') as f:
    f.write(predictor_code)

print("✅ Production predictor class saved!")

✅ Production predictor class saved!


In [126]:
import models.nba_draft_predictor as ndp

predictor = ndp.NBADraftPredictor(
    "models/nba_draft_ensemble_model_20250925.pkl", "models/nba_draft_preprocessors_20250925.pkl")


# Test with a single player (example data)
test_player = {
    'player_name': 'Test Player',
    'conf': 'ACC',
    'yr': 'Jr',
    'role_position': 'SF',
    'Min_per': 32.5,
    'usg': 28.2,
    'GP': 30,
    'eFG': 0.58,
    'TS_per': 0.62,
    'FT_per': 0.85,
    'twoP_per': 0.55,
    'TP_per': 0.38,
    'ast/tov': 1.8,
    'Ortg': 115.0,
    'bpm': 7.2,
    'obpm': 5.8,
    'dbpm': 1.4,
    'adjoe': 118.0,
    'drtg': 98.0,
    'adrtg': 95.0,
    'ORB_per': 8.5,
    'DRB_per': 18.2,
    'AST_per': 22.0,
    'TO_per': 12.5,
    'blk_per': 3.2,
    'stl_per': 2.8,
    'ftr': 0.42,
    'rimmade/(rimmade+rimmiss)': 0.68,
    'midmade/(midmade+midmiss)': 0.45,
    'dunksmade/(dunksmade+dunksmiss)': 0.85,
    'pts': 18.5,
    'treb': 8.2,
    'ast': 4.1,
    'stl': 1.2,
    'blk': 1.8
}

# Make prediction
result = predictor.predict_draft_probability(test_player)
print("\n🏀 PREDICTION RESULT:")
print(f"   Player: {result['player_name']}")
print(f"   Draft Probability: {result['draft_probability']:.1%}")
print(f"   Prediction: {result['prediction']}")
print(f"   Confidence: {result['confidence']}")

✅ NBA Draft Predictor loaded successfully!
   Model: VotingClassifier
   Expected features: 40

🏀 PREDICTION RESULT:
   Player: Test Player
   Draft Probability: 54.3%
   Prediction: Drafted
   Confidence: Medium


c:\Users\ASUS\Documents\Side Missions\scout-ai\backend\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ASUS\Documents\Side Missions\scout-ai\backend\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
